In [5]:
import os
import requests
import time 

from tqdm import tqdm

In [1]:
# setting download parameters
sba = 'NW' # Other subareas (Table 3)
var = 'SST' # variable name 
comp = 'month' # monthly composite | here can be 'day', 'month' or 'year' 
init = 'A' # Initial character of the sensor name, MODIS-Aqua (A) and SGLI-GCOMC (GS) 
ys = 2002 # start year
ms = 1 # start month
ds = 1 # start day of month
ye = 2025 # end year + 1, if end is 1997 then use yend = 1997 + 1 
me = 13 # end month + 1, if end is 12 (Dec) then use 13, if 1 then use 2
de = 30 # end day
file_ext = ('nc', 'png') # file extension (type) to download, ('nc', 'png') or ('nc',) or ('png',) 
url = 'https://ocean.nowpap3.go.jp/image_search/{filetype}/{subarea}/{year}/{filename}'

In [2]:
# Day month fetching file generator
def daymonth_filegen(filetype:tuple=file_ext):
    # Define the netCDF (PNG) file name
    for month in range(ms, me):
        if comp == 'day':
            for day in range(ds, de):
                files = [f'{init}{year}{month:02}{day:02}_{var}_{sba}_{comp}.{ext}' 
                         for ext in filetype]
                
                yield from [url.format(filetype='netcdf', subarea=sba, year=year, filename=f) 
                            if f.endswith('.nc') else 
                            url.format(filetype='images', subarea=sba, year=year, filename=f)
                            for f in files]
                            
        if comp == 'month':
            files = [f'{init}{year}{month:02}_{var}_{sba}_{comp}.{ext}'
                     for ext in filetype]
            
            yield from [url.format(filetype='netcdf', subarea=sba, year=year, filename=f) 
                        if f.endswith('.nc') else 
                        url.format(filetype='images', subarea=sba, year=year, filename=f)
                        for f in files]

In [3]:
# Function to download the data
def get_file(query_url:str):
    getfile =  "results/Modis/sst/"+os.path.basename(query_url)
    with requests.get(query) as r:
        if r.status_code != 200:
            print(f'{os.path.basename(query_url)}: FileNotFound')
            return
        total = int(r.headers.get('content-length'))                
        print('File: {} '.format(getfile), end='')
        with tqdm(total=total) as bar, open(getfile, "wb") as handle:
            for chunk in r.iter_content(chunk_size=max(int(total / 1000), 1024 * 1024)):
                # download progress check tqdm
                if chunk: 
                    handle.write(chunk)
                    time.sleep(0.1)
                    bar.update(len(chunk))

In [6]:
# Now retrieve the data from NMEW
for year in range(ys, ye):
    print(f'{year}...')
    if comp in ('day', 'month'):
        for query in daymonth_filegen():
            # ----------------------
            get_file(query_url=query)
            # ----------------------
            
    if comp == 'year':
        ncfile = f'{init}{year}_{var}_{sba}_{comp}.nc'
        query = url.format(filetype='netcdf', subarea=sba, year=year, filename=ncfile)
        # ----------------------
        get_file(query_url=query)
        # ----------------------

        pngfile = f'{init}{year}_{var}_{sba}_{comp}.png'
        query = url.format(filetype='images', subarea=sba, year=year, filename=pngfile)
        # ----------------------
        get_file(query_url=query)
        # ----------------------

print('done!')

2002...
A200201_SST_NW_month.nc: FileNotFound
A200201_SST_NW_month.png: FileNotFound
A200202_SST_NW_month.nc: FileNotFound
A200202_SST_NW_month.png: FileNotFound
A200203_SST_NW_month.nc: FileNotFound
A200203_SST_NW_month.png: FileNotFound
A200204_SST_NW_month.nc: FileNotFound
A200204_SST_NW_month.png: FileNotFound
A200205_SST_NW_month.nc: FileNotFound
A200205_SST_NW_month.png: FileNotFound
A200206_SST_NW_month.nc: FileNotFound
A200206_SST_NW_month.png: FileNotFound
File: results/Modis/sst/A200207_SST_NW_month.nc 

100%|███████████████████████████████████████████████████| 3616709/3616709 [00:00<00:00, 8741841.12it/s]


File: results/Modis/sst/A200207_SST_NW_month.png 

100%|███████████████████████████████████████████████████| 2745860/2745860 [00:00<00:00, 8593267.12it/s]


File: results/Modis/sst/A200208_SST_NW_month.nc 

...

File: results/Modis/sst/A202309_SST_NW_month.nc 

100%|███████████████████████████████████████████████████| 1159327/1159327 [00:00<00:00, 5704965.01it/s]


File: results/Modis/sst/A202309_SST_NW_month.png 

100%|███████████████████████████████████████████████████| 1715742/1715742 [00:00<00:00, 7894579.63it/s]


A202310_SST_NW_month.nc: FileNotFound
A202310_SST_NW_month.png: FileNotFound
A202311_SST_NW_month.nc: FileNotFound
A202311_SST_NW_month.png: FileNotFound
A202312_SST_NW_month.nc: FileNotFound
A202312_SST_NW_month.png: FileNotFound
2024...
A202401_SST_NW_month.nc: FileNotFound
A202401_SST_NW_month.png: FileNotFound
A202402_SST_NW_month.nc: FileNotFound
A202402_SST_NW_month.png: FileNotFound
A202403_SST_NW_month.nc: FileNotFound
A202403_SST_NW_month.png: FileNotFound
A202404_SST_NW_month.nc: FileNotFound
A202404_SST_NW_month.png: FileNotFound
A202405_SST_NW_month.nc: FileNotFound
A202405_SST_NW_month.png: FileNotFound
A202406_SST_NW_month.nc: FileNotFound
A202406_SST_NW_month.png: FileNotFound
A202407_SST_NW_month.nc: FileNotFound
A202407_SST_NW_month.png: FileNotFound
A202408_SST_NW_month.nc: FileNotFound
A202408_SST_NW_month.png: FileNotFound
A202409_SST_NW_month.nc: FileNotFound
A202409_SST_NW_month.png: FileNotFound
A202410_SST_NW_month.nc: FileNotFound
A202410_SST_NW_month.png: File